### Demo: converting OPHYS to NWB and then extracting relevant metadata for analysis
Neurodata Without Borders (NWB) is a powerful tool for organizing your data for purposes such as:
* data sharing
* result reproduction
* experiment organization

Here is a demo showing its utility:
_____

### Relevant packages
For Ophys, we must have the following packages:
* `caiman`
* `decode_lab_code`

In [1]:
from datetime import datetime
from uuid import uuid4

import numpy as np

from dateutil import tz

from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.behavior import Position, SpatialSeries
from pynwb.epoch import TimeIntervals
from pynwb.file import Subject

from decode_lab_code.preprocessing.ophys.caiman_wrapper import caiman_preprocess

### Lets build our NWB file using the ``NWBFile`` object

In [2]:
session_start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz("US/Pacific"))
# initialize the nwbfile
nwbfile = NWBFile(
    session_description=input("Enter description of your recording session: "),  # required
    identifier=str(uuid4()),  # required
    session_start_time=session_start_time,  # required
    session_id=input("Enter unique identifier for session: "),  # optional
    experimenter=[
        input("Enter experimenter name: "),
    ],  # optional
    lab=input("Enter lab name: "),  # optional
    institution=input("Enter institution name: "),  # optional
    experiment_description=input("Enter a description of your experiment"),  # optional
    related_publications=input("Enter any information about publication (if relevant)"),  # optional
)

### Enter information about subject

In [3]:
# enter information about subject
nwbfile.subject = Subject(
    subject_id=input("Enter subject ID: "),
    age=input("Enter subject age as such (PD100):  "),
    description=input("Enter subject identifier: "),
    species=input("Enter species name: "),
    sex=input("Enter sex of subject: "),
)

### Add our data to the NWB file

Interface with the user to get some information about directory

In [4]:
# directory information
folder_name = input("Enter the folder name for your data: ")
fname_neuron = input("Enter file name with extension: ")
frame_rate = float(input("Enter the frame rate: "))

By initializing the `caiman_process` object, we can then access the attribute `get_frames`, which allows us to directly extract the recorded video

In [5]:
# if you get the Error: "Exception: A cluster is already runnning", restart the kernel
cp = caiman_preprocess(folder_name,fname_neuron,frame_rate,False) # False flag for preventing cluster

# this wrapper function interfaces directly with caiman to extract our video
data = cp.get_frames()
cp.watch_movie()

Loading movie


100%|██████████| 1/1 [00:00<00:00,  3.59it/s]


Lets view the video for fun

### Now lets add the video to the NWB file

In [6]:
time_series_with_rate = TimeSeries(
    name="ophys",
    data=data,
    unit="pixels",
    starting_time=0.0,
    # I'm not sure if this is numsamples/sec or sec/numsamples
    rate=frame_rate, # sampled every second (make sure this is correct***)
)
time_series_with_rate
nwbfile.add_acquisition(time_series_with_rate)

### We can save out our data

In [7]:
# write
with NWBHDF5IO(folder_name+"/data_nwb.nwb", "w") as io:
    io.write(nwbfile)

### Lets use nwbwidgets to see our data

In [10]:
import nwbwidgets
nwbwidgets.nwb2widget(nwbfile)

ValueError: Visualization for TimeSeries that has data with shape (514, 512, 512) not implemented

ValueError: Visualization for TimeSeries that has data with shape (514, 512, 512) not implemented